<a href="https://colab.research.google.com/github/jellyXuuuuu/CovidNetDeepLearning/blob/main/test_for_covidNe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# #import necessary libraries
# import tensorflow as tf

# #load training data and split into train and test sets
# mnist = tf.keras.datasets.mnist
 
# (x_train,y_train), (x_test,y_test) = mnist.load_data()
# x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
# #define model
# model = tf.keras.models.Sequential([
#                                tf.keras.layers.Flatten(input_shape=(28,28)),
#                                    tf.keras.layers.Dense(128,activation='relu'),
#                                    tf.keras.layers.Dropout(0.2),
#                                    tf.keras.layers.Dense(10)
# ])

# #define loss function variable
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# #define optimizer,loss function and evaluation metric
# model.compile(optimizer='adam',
#              loss=loss_fn,
#              metrics=['accuracy'])

# #train the model
# model.fit(x_train,y_train,epochs=5)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2985 - accuracy: 0.9130
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1443 - accuracy: 0.9571
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1071 - accuracy: 0.9676
Epoch 4/5
1242/1875 [==================>...........] - ETA: 1s - loss: 0.0905 - accuracy: 0.9718

# Set Up
## ricord data set 

In [1]:
!pip3 install python-gdcm
!pip3 install pydicom
# !conda install -c conda-forge pillow -y
# !conda install -c conda-forge pydicom -y
# !conda install -c conda-forge gdcm -y

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.0 MB 24.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 29.0 MB/s 


In [2]:
import os
import cv2
import glob
import pydicom
# import dicom
from pydicom.pixel_data_handlers import apply_modality_lut, apply_voi_lut
import numpy as np
import pandas as pd

In [3]:
"""Helper functions"""
def _extract_data(df_row):
    return df_row['Anon MRN'], df_row['Anon TCIA Study Date'], df_row['Anon Exam Description'], df_row['Anon Study UID']


def load_ricord_metadata(ricord_meta_file):
    df = pd.read_excel(ricord_meta_file, sheet_name='CR Pos - TCIA Submission')
    ricord_metadata = []
    for index, row in df.iterrows():
        ricord_metadata.append(_extract_data(row))
    return ricord_metadata

def make_ricord_dict(ricord_data_set_file):
    """Loads bboxes from the given text file"""
    ricord_dict = {}
    with open(ricord_data_set_file, 'r') as f:
        for line in f.readlines():
            # Values after file name are crop dimensions
            if(len(line.split()) > 1):
                fname, xmin, ymin, xmax, ymax = line.rstrip('\n').split()
                bbox = tuple(int(c) for c in (xmin, ymin, xmax, ymax))
                ricord_dict[fname] = bbox
            else:
                fname = line.rstrip('\n')
                ricord_dict[fname] = None
                
    return ricord_dict

In [5]:
"""
RICORD data requires some preprocessing before splitting into test/train. 
Some images contain padding and some images are unusable. 
ricord_data_set.txt contains the name of usable images along with bounding box dimensions if needed. 

This cell crops the DICOM according to dimensions in ricord_data_set.txt and saves the image 
as png format in out_dir.

DICOM_images and "MIDRC-RICORD-1c Clinical Data Jan 13 2021 .xlsx" need to be downloaded from
https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=70230281 before running this cell
"""

# ricord_dir = 'path/to/DICOM_images'
# ricord_meta_file = 'path/to/MIDRC-RICORD-1c Clinical Data Jan 13 2021 .xlsx'
# ricord_dir = 'G:\\dataset\DICOM\\manifest-1610656454899\MIDRC-RICORD-1C\\MIDRC-RICORD-1C-419639-000025\05-03-2000-NA-XR CHEST 1 VIEW AP-56001\\1.000000-AP-79621'
ricord_dir = "drive/MyDrive/covid/DICOM/manifest-1610656454899"
ricord_meta_file = 'drive/MyDrive/covid/DICOM/MIDRC-RICORD-1c Clinical Data Jan 13 2021 .xlsx'

out_dir = 'ricord_images'
ricord_set_file = 'drive/MyDrive/covid/ricord_data_set.txt'

os.makedirs(out_dir, exist_ok=True)

ricord_dict = make_ricord_dict(ricord_set_file)

metadata = load_ricord_metadata(ricord_meta_file)
file_count = 0
for mrn, date, desc, uid in metadata:
    date = date.strftime('%m-%d-%Y')
    uid = uid[-5:]
    study_dir = os.path.join(ricord_dir, 'MIDRC-RICORD-1C-{}'.format(mrn), '*-{}'.format(uid))
#     patht = study_dir + '*' + '*.dcm'
    patht2 = os.path.join(study_dir, '*', '*.dcm')
    newfile = glob.glob(patht2)
    
    dcm_files = sorted(glob.glob(os.path.join(study_dir, "*", "*.dcm")))
    # print("patht2", patht2)
    # print("study_dir", study_dir)
    # print("newfile", newfile)
    # print("dcm_files", dcm_files)
    
#     for infile in glob.glob(os.path.join(study_dir, '*', '*.dcm')):
#         print(infile)
    for i, dcm_file in enumerate(dcm_files):
        # Create output path and check if image is to be included
        out_fname = 'MIDRC-RICORD-1C-{}-{}-{}.png'.format(mrn, uid, i)
        out_path = os.path.join(out_dir, out_fname)

        if out_fname not in ricord_dict:
            continue

        # Load DICOM image
        ds = pydicom.dcmread(dcm_file)

        # Verify orientation
        if ds.ViewPosition != 'AP' and ds.ViewPosition != 'PA':
            print('Image from MRN-{} Date-{} UID-{} in position {}'.format(mrn, date, uid, ds.ViewPosition))
            continue

        # Apply transformations if required
        if ds.pixel_array.dtype != np.uint8:
            # Apply LUT transforms
            arr = apply_modality_lut(ds.pixel_array, ds)
            if arr.dtype == np.float64 and ds.RescaleSlope == 1 and ds.RescaleIntercept == 0:
                arr = arr.astype(np.uint16)
            arr = apply_voi_lut(arr, ds)
            arr = arr.astype(np.float64)

            # Normalize to [0, 1]
            arr = (arr - arr.min())/arr.ptp()

            # Invert MONOCHROME1 images
            if ds.PhotometricInterpretation == 'MONOCHROME1':
                arr = 1. - arr

            # Convert to uint8
            image = np.uint8(255.*arr)
        else:
            # Invert MONOCHROME1 images
            if ds.PhotometricInterpretation == 'MONOCHROME1':
                image = 255 - ds.pixel_array
            else:
                image = ds.pixel_array

        # Crop if necessary
        bbox = ricord_dict[out_fname]
        if bbox is not None:
            image = image[bbox[1]:bbox[3], bbox[0]:bbox[2]]

        # Save image
        cv2.imwrite(out_path, image)
        file_count += 1
print('Created {} files'.format(file_count))

patht2 drive/MyDrive/DICOM/manifest-1610656454899/MIDRC-RICORD-1C-419639-000002/*-10161/*/*.dcm
study_dir drive/MyDrive/DICOM/manifest-1610656454899/MIDRC-RICORD-1C-419639-000002/*-10161
newfile []
dcm_files []
patht2 drive/MyDrive/DICOM/manifest-1610656454899/MIDRC-RICORD-1C-419639-000025/*-53641/*/*.dcm
study_dir drive/MyDrive/DICOM/manifest-1610656454899/MIDRC-RICORD-1C-419639-000025/*-53641
newfile []
dcm_files []
patht2 drive/MyDrive/DICOM/manifest-1610656454899/MIDRC-RICORD-1C-419639-000025/*-56001/*/*.dcm
study_dir drive/MyDrive/DICOM/manifest-1610656454899/MIDRC-RICORD-1C-419639-000025/*-56001
newfile []
dcm_files []
patht2 drive/MyDrive/DICOM/manifest-1610656454899/MIDRC-RICORD-1C-419639-000025/*-50888/*/*.dcm
study_dir drive/MyDrive/DICOM/manifest-1610656454899/MIDRC-RICORD-1C-419639-000025/*-50888
newfile []
dcm_files []
patht2 drive/MyDrive/DICOM/manifest-1610656454899/MIDRC-RICORD-1C-419639-000025/*-56207/*/*.dcm
study_dir drive/MyDrive/DICOM/manifest-1610656454899/MIDRC-R

In [ ]:
# unzip dataset zip
!unzip rsna-pneumonia-detection-challenge.zip

unzip:  cannot find or open rsna-pneumonia-detection-challenge.zip, rsna-pneumonia-detection-challenge.zip.zip or rsna-pneumonia-detection-challenge.zip.ZIP.


## create covid x

In [ ]:
# unzip dataset zip
!unzip covid-chestxray-dataset.zip

In [6]:
import numpy as np
import pandas as pd
import os
import random 
from shutil import copyfile
import pydicom as dicom
import cv2
import glob

In [7]:
# set parameters here
savepath = 'data'
seed = 0
np.random.seed(seed) # Reset the seed so all runs are the same.
random.seed(seed)
MAXVAL = 255  # Range [0 255]

# path to covid-19 dataset from https://github.com/ieee8023/covid-chestxray-dataset
# cohen_imgpath = './covid-chestxray-dataset/images' 
# cohen_csvpath = './covid-chestxray-dataset/metadata.csv'
cohen_imgpath = 'G:\\dataset\\covid-chestxray-dataset\\images' 
cohen_csvpath = 'G:\\dataset\\covid-chestxray-dataset\\metadata.csv'

# path to covid-19 dataset from https://github.com/agchung/Figure1-COVID-chestxray-dataset
# fig1_imgpath = './Figure1-COVID-chestxray-dataset/images'
# fig1_csvpath = './Figure1-COVID-chestxray-dataset/metadata.csv'
fig1_imgpath = 'G:\\dataset\\Figure1-COVID-chestxray-dataset\\images'
fig1_csvpath = 'G:\\dataset\\Figure1-COVID-chestxray-dataset\\metadata.csv'

# path to covid-19 dataset from https://github.com/agchung/Actualmed-COVID-chestxray-dataset
# actmed_imgpath = './Actualmed-COVID-chestxray-dataset/images'
# actmed_csvpath = './Actualmed-COVID-chestxray-dataset/metadata.csv'
actmed_imgpath = 'G:\\dataset\\Actualmed-COVID-chestxray-dataset\\images'
actmed_csvpath = 'G:\\dataset\\Actualmed-COVID-chestxray-dataset\\metadata.csv'

# path to covid-19 dataset from https://www.kaggle.com/tawsifurrahman/covid19-radiography-database
# sirm_imgpath = './COVID-19-Radiography-Database/COVID'
# sirm_csvpath = './COVID-19-Radiography-Database/COVID.metadata.xlsx'
sirm_imgpath = 'G:\\dataset\\COVID-19-Radiography-Database\\COVID\\images'
sirm_csvpath = 'G:\\dataset\\COVID-19-Radiography-Database\\COVID.metadata.xlsx'

# path to https://www.kaggle.com/c/rsna-pneumonia-detection-challenge
# rsna_datapath = './rsna-pneumonia-detection-challenge'
rsna_datapath = 'G:\\dataset\\rsna-pneumonia-detection-challenge'
# get all the normal from here
rsna_csvname = 'stage_2_detailed_class_info.csv' 
# get all the 1s from here since 1 indicate pneumonia
# found that images that aren't pneunmonia and also not normal are classified as 0s
rsna_csvname2 = 'stage_2_train_labels.csv' 
rsna_imgpath = 'stage_2_train_images'

# path to ricord covid-19 images created by create_ricord_dataset/create_ricord_dataset.ipynb
# run create_ricord_dataset.ipynb before this notebook
ricord_imgpath = 'create_ricord_dataset/ricord_images'
ricord_txt = 'create_ricord_dataset/ricord_data_set.txt'

# parameters for COVIDx dataset
train = []
test = []
test_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
train_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}

mapping = dict()
mapping['COVID-19'] = 'COVID-19'
mapping['SARS'] = 'pneumonia'
mapping['MERS'] = 'pneumonia'
mapping['Streptococcus'] = 'pneumonia'
mapping['Klebsiella'] = 'pneumonia'
mapping['Chlamydophila'] = 'pneumonia'
mapping['Legionella'] = 'pneumonia'
mapping['E.Coli'] = 'pneumonia'
mapping['Normal'] = 'normal'
mapping['Lung Opacity'] = 'pneumonia'
mapping['1'] = 'pneumonia'

# train/test split
split = 0.1

# to avoid duplicates
patient_imgpath = {}


#%% [code]
# adapted from https://github.com/mlmed/torchxrayvision/blob/master/torchxrayvision/datasets.py#L814
cohen_csv = pd.read_csv(cohen_csvpath, nrows=None)
#idx_pa = csv["view"] == "PA"  # Keep only the PA view
views = ["PA", "AP", "AP Supine", "AP semi erect", "AP erect"]
cohen_idx_keep = cohen_csv.view.isin(views)
cohen_csv = cohen_csv[cohen_idx_keep]

fig1_csv = pd.read_csv(fig1_csvpath, encoding='ISO-8859-1', nrows=None)
actmed_csv = pd.read_csv(actmed_csvpath, nrows=None)

sirm_csv = pd.read_excel(sirm_csvpath)


FileNotFoundError: ignored

In [ ]:

Code cell <undefined>
#%% [code]
# get non-COVID19 viral, bacteria, and COVID-19 infections from covid-chestxray-dataset, figure1 and actualmed
# stored as patient id, image filename and label
filename_label = {'normal': [], 'pneumonia': [], 'COVID-19': []}
count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
covid_ds = {'cohen': [], 'fig1': [], 'actmed': [], 'sirm': [], 'ricord': []}

for index, row in cohen_csv.iterrows():
    f = row['finding'].split('/')[-1] # take final finding in hierarchy, for the case of COVID-19, ARDS
    if f == 'COVID-19' and ('eurorad.org' in row['url'] or 'ml-workgroup' in row['url'] or 'sirm.org' in row['url']):
        # skip COVID-19 positive images from eurorad to not duplicate sirm images
        pass
    elif f in mapping: # 
        count[mapping[f]] += 1
        entry = [str(row['patientid']), row['filename'], mapping[f], 'cohen']
        filename_label[mapping[f]].append(entry)
        if mapping[f] == 'COVID-19':
            covid_ds['cohen'].append(str(row['patientid']))
        
for index, row in fig1_csv.iterrows():
    if not str(row['finding']) == 'nan':
        f = row['finding'].split(',')[0] # take the first finding
        if f in mapping: # 
            count[mapping[f]] += 1
            if os.path.exists(os.path.join(fig1_imgpath, row['patientid'] + '.jpg')):
                entry = [row['patientid'], row['patientid'] + '.jpg', mapping[f], 'fig1']
            elif os.path.exists(os.path.join(fig1_imgpath, row['patientid'] + '.png')):
                entry = [row['patientid'], row['patientid'] + '.png', mapping[f], 'fig1']
            filename_label[mapping[f]].append(entry)
            if mapping[f] == 'COVID-19':
                covid_ds['fig1'].append(row['patientid'])

for index, row in actmed_csv.iterrows():
    if not str(row['finding']) == 'nan':
        f = row['finding'].split(',')[0]
        if f in mapping:
            count[mapping[f]] += 1
            entry = [row['patientid'], row['imagename'], mapping[f], 'actmed']
            filename_label[mapping[f]].append(entry)
            if mapping[f] == 'COVID-19':
                covid_ds['actmed'].append(row['patientid'])
    
sirm = set(sirm_csv['URL'])
cohen = set(cohen_csv['url'])
cohen.add('https://github.com/ieee8023/covid-chestxray-dataset')  # Add base URL to remove sirm images from ieee dataset
discard = ['100', '101', '102', '103', '104', '105', 
           '110', '111', '112', '113', '122', '123', 
           '124', '125', '126', '217']

for idx, row in sirm_csv.iterrows():
    patientid = row['FILE NAME']
    if row['URL'] not in cohen and patientid[patientid.find('(')+1:patientid.find(')')] not in discard:
        count[mapping['COVID-19']] += 1
        imagename = patientid + '.' + row['FORMAT'].lower()
        if not os.path.exists(os.path.join(sirm_imgpath, imagename)):
#             print('1', imagename.rsplit(".png"))
#             ans = imagename.rsplit(".png")[0]
#             print('2', ans)
#             ans2 = ans.split("COVID-")
#             print('3', ans2)
#             ans4 = ans2[1]
#             print('4', ans4)
            imagename = "COVID ({}).png".format(imagename.rsplit(".png")[0].split("COVID-")[1])
        entry = [patientid, imagename, mapping['COVID-19'], 'sirm']
        filename_label[mapping['COVID-19']].append(entry)
        covid_ds['sirm'].append(patientid)

# get ricord file names 
with open(ricord_txt) as f:
    ricord_file_names = [line.split()[0] for line in f]

for imagename in ricord_file_names:
    count[mapping['COVID-19']] += 1 # since RICORD data is all COVID-19 postive images
    patientid = imagename.split('-')[3] + '-' + imagename.split('-')[4]
    
    entry = [patientid, imagename, mapping['COVID-19'], 'ricord']
    filename_label[mapping['COVID-19']].append(entry)
    
    covid_ds['ricord'].append(patientid)
    
print('Data distribution from covid datasets:')
print(count)

In [ ]:
# Create list of RICORD patients to be added to test, equal to 200 images

# We want to prevent patients present in both train and test
# Get list of patients who have one image
ricord_patients = []
for label in filename_label['COVID-19']:
    if label[3] == 'ricord':
        ricord_patients.append(label[0])

pt_with_one_image = [x for x in ricord_patients if ricord_patients.count(x) == 1] # contrains 176 patients
print(len(pt_with_one_image))
Execution output
0KB
	Stream
		176

Code cell <undefined>
#%% [code]
# add covid-chestxray-dataset, figure1 and actualmed into COVIDx dataset
# since these datasets don't have test dataset, split into train/test by patientid
# for covid-chestxray-dataset:
# patient 8 is used as non-COVID19 viral test
# patient 31 is used as bacterial test
# patients 19, 20, 36, 42, 86 are used as COVID-19 viral test
# for figure 1:
# patients 24, 25, 27, 29, 30, 32, 33, 36, 37, 38

ds_imgpath = {'cohen': cohen_imgpath, 'fig1': fig1_imgpath, 'actmed': actmed_imgpath, 'sirm': sirm_imgpath, 'ricord':ricord_imgpath}

for key in filename_label.keys():
    arr = np.array(filename_label[key])
    if arr.size == 0:
        continue
    # split by patients
    # num_diff_patients = len(np.unique(arr[:,0]))
    # num_test = max(1, round(split*num_diff_patients))
    # select num_test number of random patients
    # random.sample(list(arr[:,0]), num_test)
    if key == 'pneumonia':
        test_patients = ['8', '31']
    elif key == 'COVID-19':
        test_patients = ['19', '20', '36', '42', '86', 
                         '94', '97', '117', '132', 
                         '138', '144', '150', '163', '169', '174', '175', '179', '190', '191'
                         'COVID-00024', 'COVID-00025', 'COVID-00026', 'COVID-00027', 'COVID-00029',
                         'COVID-00030', 'COVID-00032', 'COVID-00033', 'COVID-00035', 'COVID-00036',
                         'COVID-00037', 'COVID-00038',
                         'ANON24', 'ANON45', 'ANON126', 'ANON106', 'ANON67',
                         'ANON153', 'ANON135', 'ANON44', 'ANON29', 'ANON201', 
                         'ANON191', 'ANON234', 'ANON110', 'ANON112', 'ANON73', 
                         'ANON220', 'ANON189', 'ANON30', 'ANON53', 'ANON46',
                         'ANON218', 'ANON240', 'ANON100', 'ANON237', 'ANON158',
                         'ANON174', 'ANON19', 'ANON195',
                         'COVID 119', 'COVID 87', 'COVID 70', 'COVID 94', 
                         'COVID 215', 'COVID 77', 'COVID 213', 'COVID 81', 
                         'COVID 216', 'COVID 72', 'COVID 106', 'COVID 131', 
                         'COVID 107', 'COVID 116', 'COVID 95', 'COVID 214', 
                         'COVID 129']
#         Add 178 RICORD patients to COVID-19, equal to 200 images
        test_patients.extend(pt_with_one_image)
        test_patients.extend(['419639-000025', '419639-001464'])
    else: 
        test_patients = []
    print('Key: ', key)
    print('Test patients: ', test_patients)
    # go through all the patients
    for patient in arr:
        if patient[0] not in patient_imgpath:
            patient_imgpath[patient[0]] = [patient[1]]
        else:
            if patient[1] not in patient_imgpath[patient[0]]:
                patient_imgpath[patient[0]].append(patient[1])
            else:
                continue  # skip since image has already been written
        if patient[0] in test_patients:
            if patient[3] == 'sirm':
                image = cv2.imread(os.path.join(ds_imgpath[patient[3]], patient[1]))
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                patient[1] = patient[1].replace(' ', '')
                cv2.imwrite(os.path.join(savepath, 'test', patient[1]), gray)
            else:
                print("testpath:", ds_imgpath[patient[3]])
                copyfile(os.path.join(ds_imgpath[patient[3]], patient[1]), os.path.join(savepath, 'test', patient[1]))
            test.append(patient)
            test_count[patient[2]] += 1
        else:
            if patient[3] == 'sirm':
#                 print("patient 1 /////", patient[1])
#                 print("path:", os.path.join(ds_imgpath[patient[3]]))
#                 print(patient[1].replace(" (","-").replace(")",""))
                image = cv2.imread(os.path.join(ds_imgpath[patient[3]], patient[1].replace(" (","-").replace(")","")))
#                 print(image)
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                patient[1] = patient[1].replace(' ', '')
                cv2.imwrite(os.path.join(savepath, 'train', patient[1]), gray)
            else:
                copyfile(os.path.join(ds_imgpath[patient[3]], patient[1]), os.path.join(savepath, 'train', patient[1]))
            train.append(patient)
            train_count[patient[2]] += 1

print('test count: ', test_count)
print('train count: ', train_count)
Execution output
8KB
	Error
		FileNotFoundError
		---------------------------------------------------------------------------
		FileNotFoundError                         Traceback (most recent call last)
		<ipython-input-12-ecbdf7db3840> in <module>
		     63             else:
		     64                 print("testpath:", ds_imgpath[patient[3]])
		---> 65                 copyfile(os.path.join(ds_imgpath[patient[3]], patient[1]), os.path.join(savepath, 'test', patient[1]))
		     66             test.append(patient)
		     67             test_count[patient[2]] += 1
		
		d:\computer\python3\lib\shutil.py in copyfile(src, dst, follow_symlinks)
		    257         os.symlink(os.readlink(src), dst)
		    258     else:
		--> 259         with open(src, 'rb') as fsrc, open(dst, 'wb') as fdst:
		    260             # macOS
		    261             if _HAS_FCOPYFILE:
		
		FileNotFoundError: [Errno 2] No such file or directory: 'create_ricord_dataset/ricord_images\\MIDRC-RICORD-1C-419639-000025-39552-1.png'
	Stream
		Key:  pneumonia
		Test patients:  ['8', '31']
		Key:  COVID-19
		Test patients:  ['19', '20', '36', '42', '86', '94', '97', '117', '132', '138', '144', '150', '163', '169', '174', '175', '179', '190', '191COVID-00024', 'COVID-00025', 'COVID-00026', 'COVID-00027', 'COVID-00029', 'COVID-00030', 'COVID-00032', 'COVID-00033', 'COVID-00035', 'COVID-00036', 'COVID-00037', 'COVID-00038', 'ANON24', 'ANON45', 'ANON126', 'ANON106', 'ANON67', 'ANON153', 'ANON135', 'ANON44', 'ANON29', 'ANON201', 'ANON191', 'ANON234', 'ANON110', 'ANON112', 'ANON73', 'ANON220', 'ANON189', 'ANON30', 'ANON53', 'ANON46', 'ANON218', 'ANON240', 'ANON100', 'ANON237', 'ANON158', 'ANON174', 'ANON19', 'ANON195', 'COVID 119', 'COVID 87', 'COVID 70', 'COVID 94', 'COVID 215', 'COVID 77', 'COVID 213', 'COVID 81', 'COVID 216', 'COVID 72', 'COVID 106', 'COVID 131', 'COVID 107', 'COVID 116', 'COVID 95', 'COVID 214', 'COVID 129', '419639-000573', '419639-001015', '419639-001070', '419639-001433', '419639-001571', '419639-001603', '419639-001629', '419639-001762', '419639-001882', '419639-001932', '419639-002043', '419639-002050', '419639-002088', '419639-002374', '419639-002376', '419639-002733', '419639-003021', '419639-003024', '419639-003027', '419639-003060', '419639-003080', '419639-003099', '419639-003116', '419639-003169', '419639-003187', '419639-003199', '419639-003203', '419639-003216', '419639-003232', '419639-003251', '419639-003319', '419639-003336', '419639-003365', '419639-003380', 'SITE2-000002', 'SITE2-000003', 'SITE2-000014', 'SITE2-000029', 'SITE2-000034', 'SITE2-000068', 'SITE2-000072', 'SITE2-000094', 'SITE2-000096', 'SITE2-000097', 'SITE2-000098', 'SITE2-000099', 'SITE2-000108', 'SITE2-000115', 'SITE2-000120', 'SITE2-000127', 'SITE2-000131', 'SITE2-000135', 'SITE2-000141', 'SITE2-000142', 'SITE2-000147', 'SITE2-000159', 'SITE2-000162', 'SITE2-000168', 'SITE2-000170', 'SITE2-000171', 'SITE2-000173', 'SITE2-000178', 'SITE2-000202', 'SITE2-000203', 'SITE2-000211', 'SITE2-000217', 'SITE2-000229', 'SITE2-000230', 'SITE2-000237', 'SITE2-000239', 'SITE2-000244', 'SITE2-000265', 'SITE2-000270', 'SITE2-000272', 'SITE2-000291', 'SITE2-000154', '419639-000002', '419639-000082', '419639-000086', '419639-000235', '419639-000299', '419639-000402', '419639-000432', '419639-000538', '419639-000582', '419639-000586', '419639-000785', '419639-000918', '419639-001007', '419639-001085', '419639-001155', '419639-001222', '419639-001287', '419639-001288', '419639-001374', '419639-001425', '419639-001505', '419639-001661', '419639-001699', '419639-001735', '419639-001782', '419639-001829', '419639-001903', '419639-002114', '419639-002281', '419639-002309', '419639-002310', '419639-002401', '419639-002463', '419639-002472', '419639-002528', '419639-002601', '419639-002724', '419639-002757', '419639-002768', '419639-002784', '419639-002871', '419639-002889', '419639-002895', '419639-002929', '419639-002997', '419639-003102', '419639-003105', '419639-003157', '419639-003186', '419639-003191', '419639-003257', '419639-003318', '419639-003334', '419639-003338', '419639-003340', '419639-003360', '419639-003371', '440808-000009', '440808-000016', '440808-000019', '440808-000020', 'SITE2-000001', 'SITE2-000015', 'SITE2-000018', 'SITE2-000023', 'SITE2-000028', 'SITE2-000035', 'SITE2-000045', 'SITE2-000048', 'SITE2-000058', 'SITE2-000066', 'SITE2-000067', 'SITE2-000105', 'SITE2-000106', 'SITE2-000116', 'SITE2-000124', 'SITE2-000125', 'SITE2-000136', 'SITE2-000149', 'SITE2-000153', 'SITE2-000155', 'SITE2-000176', 'SITE2-000179', 'SITE2-000181', 'SITE2-000182', 'SITE2-000185', 'SITE2-000188', 'SITE2-000200', 'SITE2-000204', 'SITE2-000206', 'SITE2-000207', 'SITE2-000208', 'SITE2-000210', 'SITE2-000220', 'SITE2-000221', 'SITE2-000235', 'SITE2-000238', 'SITE2-000279', 'SITE2-000283', 'SITE2-000297', '419639-000025', '419639-001464']
		testpath: create_ricord_dataset/ricord_images

Code cell <undefined>
#%% [code]
# add normal and rest of pneumonia cases from https://www.kaggle.com/c/rsna-pneumonia-detection-challenge
csv_normal = pd.read_csv(os.path.join(rsna_datapath, rsna_csvname), nrows=None)
csv_pneu = pd.read_csv(os.path.join(rsna_datapath, rsna_csvname2), nrows=None)
patients = {'normal': [], 'pneumonia': []}

for index, row in csv_normal.iterrows():
    if row['class'] == 'Normal':
        patients['normal'].append(row['patientId'])

for index, row in csv_pneu.iterrows():
    if int(row['Target']) == 1:
        patients['pneumonia'].append(row['patientId'])

for key in patients.keys():
    arr = np.array(patients[key])
    if arr.size == 0:
        continue
    # split by patients 
    # num_diff_patients = len(np.unique(arr))
    # num_test = max(1, round(split*num_diff_patients))
    test_patients = np.load('rsna_test_patients_{}.npy'.format(key)) # random.sample(list(arr), num_test), download the .npy files from the repo.
    # np.save('rsna_test_patients_{}.npy'.format(key), np.array(test_patients))
    for patient in arr:
        if patient not in patient_imgpath:
            patient_imgpath[patient] = [patient]
        else:
            continue  # skip since image has already been written
                
        ds = dicom.dcmread(os.path.join(rsna_datapath, rsna_imgpath, patient + '.dcm'))
        pixel_array_numpy = ds.pixel_array
        imgname = patient + '.png'
        if patient in test_patients:
            cv2.imwrite(os.path.join(savepath, 'test', imgname), pixel_array_numpy)
            test.append([patient, imgname, key, 'rsna'])
            test_count[key] += 1
        else:
            cv2.imwrite(os.path.join(savepath, 'train', imgname), pixel_array_numpy)
            train.append([patient, imgname, key, 'rsna'])
            train_count[key] += 1

print('test count: ', test_count)
print('train count: ', train_count)
Execution output
0KB
	Stream
		test count:  {'normal': 100, 'pneumonia': 105, 'COVID-19': 74}
		train count:  {'normal': 8751, 'pneumonia': 5964, 'COVID-19': 3753}

Code cell <undefined>
#%% [code]
# final stats
print('Final stats')
print('Train count: ', train_count)
print('Test count: ', test_count)
print('Total length of train: ', len(train))
print('Total length of test: ', len(test))
Execution output
0KB
	Stream
		Final stats
		Train count:  {'normal': 8751, 'pneumonia': 5964, 'COVID-19': 3753}
		Test count:  {'normal': 100, 'pneumonia': 105, 'COVID-19': 74}
		Total length of train:  18468
		Total length of test:  279

Code cell <undefined>
#%% [code]
# export to train and test csv
# format as patientid, filename, label, separated by a space
train_file = open("train_split.txt",'w') 
for sample in train:
    if len(sample) == 4:
        info = str(sample[0]) + ' ' + sample[1] + ' ' + sample[2] + ' ' + sample[3] + '\n'
    else:
        info = str(sample[0]) + ' ' + sample[1] + ' ' + sample[2] + '\n'
    train_file.write(info)

train_file.close()

test_file = open("test_split.txt", 'w')
for sample in test:
    if len(sample) == 4:
        info = str(sample[0]) + ' ' + sample[1] + ' ' + sample[2] + ' ' + sample[3] + '\n'
    else:
        info = str(sample[0]) + ' ' + sample[1] + ' ' + sample[2] + '\n'
    test_file.write(info)

test_file.close()

# Experiment